In [1]:
import ccxt
import pandas as pd
import ta
import time
from datetime import datetime
import os
import csv
from concurrent.futures import ThreadPoolExecutor
import discord
from discord import Intents
import asyncio

In [2]:
# Configurações do bot
DISCORD_TOKEN = ''
CHANNEL_ID = ''  # Insira o ID do canal

intents = discord.Intents.default()
client = discord.Client(intents=intents)

@client.event
async def on_ready():
    print(f'Bot conectado como {client.user}')

async def send_discord_notification(message):
    channel = client.get_channel(CHANNEL_ID)
    if channel:
        await channel.send(message)
    else:
        print("Canal não encontrado")

def start_discord_bot():
    loop = asyncio.get_event_loop()
    loop.create_task(client.start(DISCORD_TOKEN))

# Iniciar o bot do Discord em segundo plano
start_discord_bot()

In [3]:
# Função para obter dados de preço em tempo real para um symbol específico
def fetch_real_time_data(exchange, symbol, timeframe, limit=100):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    return data

In [4]:
# Função para calcular indicadores técnicos
def calculate_indicators(data):
    data['RSI'] = ta.momentum.RSIIndicator(data['close'], window=14).rsi()
    macd = ta.trend.MACD(data['close'])
    data['MACD'] = macd.macd()
    data['MACD_Signal'] = macd.macd_signal()
    data['SMA_50'] = ta.trend.SMAIndicator(data['close'], window=50).sma_indicator()
    data['SMA_200'] = ta.trend.SMAIndicator(data['close'], window=200).sma_indicator()
    bb = ta.volatility.BollingerBands(data['close'], window=20)
    data['Upper_BB'] = bb.bollinger_hband()
    data['Middle_BB'] = bb.bollinger_mavg()
    data['Lower_BB'] = bb.bollinger_lband()
    return data

```
# Detectar divergência entre RSI e preço
def detect_divergence(data):
    divergences = []
    for i in range(1, len(data) - 1):
        if data['close'].iloc[i] < data['close'].iloc[i - 1] and data['RSI'].iloc[i] > data['RSI'].iloc[i - 1]:
            divergences.append(('Bullish', data.index.iloc[i]))
        elif data['close'].iloc[i] > data['close'].iloc[i - 1] and data['RSI'].iloc[i] < data['RSI'].iloc[i - 1]:
            divergences.append(('Bearish', data.index.iloc[i]))
    return divergences

In [5]:
def detect_divergence(data, symbol):
    divergences = []
    log_data = []  # Lista para armazenar os dados de depuração
    threshold_timestamp = pd.Timestamp('2024-06-10 00:00:00')
    
    for i in range(1, len(data) - 1):
        # Verificar se o Timestamp atual é maior ou igual ao threshold
        if data.index[i] >= threshold_timestamp:
            # Verificar se os valores de preço e RSI não são 'nan'
            if pd.notna(data['close'].iloc[i]) and pd.notna(data['RSI'].iloc[i]):
                for j in range(i):
                    if pd.notna(data['close'].iloc[j]) and pd.notna(data['RSI'].iloc[j]):
                        # Divergência de alta
                        if data['RSI'].iloc[i] > data['RSI'].iloc[j] and data['close'].iloc[i] < data['close'].iloc[j]:
                            divergences.append(('Bullish', data.index[i]))
                            message_test = (f"Symbol: {symbol}, "
                                            f"Timestamp antigo: {data.index[j]}, Preco antigo: {data['close'].iloc[j]}, RSI antigo: {data['RSI'].iloc[j]}, "
                                            f"Timestamp atual: {data.index[i]}, Preco atual: {data['close'].iloc[i]}, RSI atual: {data['RSI'].iloc[i]}")
                            print(message_test)
                            break  # Encontrou uma divergência, não precisa continuar verificando para este índice

                        # Divergência de baixa
                        elif data['RSI'].iloc[i] < data['RSI'].iloc[j] and data['close'].iloc[i] > data['close'].iloc[j]:
                            divergences.append(('Bearish', data.index[i]))
                            break  # Encontrou uma divergência, não precisa continuar verificando para este índice

                # Armazenar os dados de depuração na lista
                log_data.append({
                    'SYMBOL': symbol,
                    'PRECO_ANTIGO': data['close'].iloc[i - 1],
                    'PRECO_ATUAL': data['close'].iloc[i],
                    'RSI_ANTIGO': data['RSI'].iloc[i - 1],
                    'RSI_ATUAL': data['RSI'].iloc[i],
                    'TIMESTAMP_ANTIGO': data.index[i - 1],
                    'TIMESTAMP_ATUAL': data.index[i]
                })

    # Converter a lista de dados de depuração em um DataFrame
    log_df = pd.DataFrame(log_data)
    
    # Definir o caminho do arquivo CSV
    file_path = 'divergence_log.csv'
    
    # Verificar se o arquivo já existe e definir o modo de escrita e cabeçalho
    write_header = not os.path.isfile(file_path)
    
    # Escrever os dados no arquivo CSV
    with open(file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['SYMBOL', 'PRECO_ANTIGO', 'PRECO_ATUAL', 'RSI_ANTIGO', 'RSI_ATUAL', 'TIMESTAMP_ANTIGO', 'TIMESTAMP_ATUAL'], delimiter=';')
        if write_header:
            writer.writeheader()
        for row in log_data:
            writer.writerow(row)
    
    return divergences


In [6]:
# Função para verificar condições de compra e venda
def check_signals(data, symbol):
    signals = []
    
    for i in range(len(data)):
        signal = {}
        signal['Date'] = data.index[i]
        signal['Close'] = data.iloc[i]['close']

        divergences = detect_divergence(data.iloc[:i + 1], symbol)  # Detectar divergências
        # Condições RSI
        #if data.iloc[i]['RSI'] < 30:
        #    signal['RSI_Signal'] = 'Buy'
        #elif data.iloc[i]['RSI'] > 70:
        #    signal['RSI_Signal'] = 'Sell'
        #else:
        #    signal['RSI_Signal'] = 'Hold'
            
        # RSI Divergence Signal
        #divergences = detect_divergence(data)
        #if divergences:
        #    last_divergence = divergences[-1]
        #    if last_divergence[0] == 'Bullish':
        #        signals['RSI_Signal'] = 'Buy'
        #    elif last_divergence[0] == 'Bearish':
        #        signals['RSI_Signal'] = 'Sell'
        #    else:
        #        signal['RSI_Signal'] = 'Hold'     


        # Condições RSI baseadas na divergência
        #divergences = detect_divergence(data.iloc[:i + 1])
        #if divergences:
        #    last_divergence = divergences.iloc[-1]
        #    if last_divergence.iloc[0] == 'Bullish':
        #        signal['RSI_Signal'] = 'Buy'
        #    elif last_divergence.iloc[0] == 'Bearish':
        #        signal['RSI_Signal'] = 'Sell'
        #else:
        #    signal['RSI_Signal'] = 'Hold'
            
        # Incorporar divergências detectadas
        #for divergence in divergences:
        #    if divergence[1] == data.index[i]:
        #        if divergence[0] == 'Bullish':
        #            signal['RSI_Signal'] = 'Buy'
        #        elif divergence[0] == 'Bearish':
        #            signal['RSI_Signal'] = 'Sell'
        #        break
        #else:
        #    signal['RSI_Signal'] = 'None'

        if divergences:
            last_divergence = divergences[-1]
            if last_divergence[0] == 'Bullish' and last_divergence[1] == data.index[i]:
                signal['RSI_Signal'] = 'Buy'
            elif last_divergence[0] == 'Bearish' and last_divergence[1] == data.index[i]:
                signal['RSI_Signal'] = 'Sell'
            else:
                signal['RSI_Signal'] = 'Hold'
        else:
            signal['RSI_Signal'] = 'None'
        
       # Imprimir valores do RSI para análise
       # print(f"Symbol: {symbol}, Timestamp: {data.index[i]}, Close Price: {data.iloc[i]['close']}, RSI: {data.iloc[i]['RSI']}, last_div: {last_divergence}")
        
        # Condições MACD
        if data.iloc[i]['MACD'] > data.iloc[i]['MACD_Signal']:
            signal['MACD_Signal'] = 'Buy'
        else:
            signal['MACD_Signal'] = 'Sell'

        # Condições Médias Móveis
        if data.iloc[i]['SMA_50'] > data.iloc[i]['SMA_200']:
            signal['SMA_Signal'] = 'Buy'
        else:
            signal['SMA_Signal'] = 'Sell'

        # Condições Bandas de Bollinger
        if data.iloc[i]['close'] < data.iloc[i]['Lower_BB']:
            signal['BB_Signal'] = 'Buy'
        elif data.iloc[i]['close'] > data.iloc[i]['Upper_BB']:
            signal['BB_Signal'] = 'Sell'
        else:
            signal['BB_Signal'] = 'Hold'

        signals.append(signal)
    return pd.DataFrame(signals)

In [7]:
def process_symbols_group(exchange, symbols):
    for symbol in symbols:
        try:
            data = fetch_real_time_data(exchange, symbol, timeframe='4h', limit=100)
            data = calculate_indicators(data)
            signals = check_signals(data, symbol)
            signals['Symbol'] = symbol  # Adicionar o nome do symbol ao DataFrame de sinais
            latest_signals = signals.tail(1)

            
            # Verificar se pelo menos 2 dos 4 sinais são 'Buy', sendo RSI obrigatoriamente 'Buy' e os outros dois não 'Sell'
            buy_count = sum(signal == 'Buy' for signal in latest_signals[['RSI_Signal', 'MACD_Signal', 'SMA_Signal', 'BB_Signal']].values.flatten())
            rsi_signal = latest_signals['RSI_Signal'].values[0]
            other_signals = latest_signals[['MACD_Signal', 'SMA_Signal', 'BB_Signal']].values.flatten()

            
            timestamp = pd.Timestamp.now()
            price = latest_signals['Close'].values[0]
            message = (f"Sinais de compra (RSI: {rsi_signal}, MACD: {latest_signals['MACD_Signal'].values[0]}, "
                       f"SMA: {latest_signals['SMA_Signal'].values[0]}, BB: {latest_signals['BB_Signal'].values[0]}) para {symbol}.\n"
                       f"Timestamp: {timestamp}\nPreço atual: {price}")
                
            #print(message)
            #if buy_count >= 2 and rsi_signal == 'Buy' and all(signal != 'Sell' for signal in other_signals):
            if rsi_signal == 'Buy':    
                timestamp = pd.Timestamp.now()
                price = latest_signals['Close'].values[0]
                message = (f"Sinais de compra (RSI: {rsi_signal}, MACD: {latest_signals['MACD_Signal'].values[0]}, "
                           f"SMA: {latest_signals['SMA_Signal'].values[0]}, BB: {latest_signals['BB_Signal'].values[0]}) para {symbol}.\n"
                           f"Timestamp: {timestamp}\nPreço atual: {price}")
                
                print(message)
                asyncio.run_coroutine_threadsafe(send_discord_notification(message), client.loop)
            
            # Verificar se pelo menos 2 dos 4 sinais são 'Sell', sendo RSI obrigatoriamente 'Sell' e os outros dois não 'Buy'
            #sell_count = sum(signal == 'Sell' for signal in latest_signals[['RSI_Signal', 'MACD_Signal', 'SMA_Signal', 'BB_Signal']].values.flatten())
            #rsi_signal_sell = latest_signals['RSI_Signal'].values[0]
            #other_signals_sell = latest_signals[['MACD_Signal', 'SMA_Signal', 'BB_Signal']].values.flatten()
            #if sell_count >= 2 and rsi_signal_sell == 'Sell' and all(signal != 'Buy' for signal in other_signals_sell):
            elif rsi_signal == 'Sell':
                timestamp_sell = pd.Timestamp.now()
                price_sell = latest_signals['Close'].values[0]
                message_sell = (f"Sinais de compra (RSI: {rsi_signal}, MACD: {latest_signals['MACD_Signal'].values[0]}, "
                                f"SMA: {latest_signals['SMA_Signal'].values[0]}, BB: {latest_signals['BB_Signal'].values[0]}) para {symbol}.\n"
                                f"Timestamp: {timestamp_sell}\nPreço atual: {price_sell}")
            
                print(message_sell)
                asyncio.run_coroutine_threadsafe(send_discord_notification(message_sell), client.loop)
        except Exception as e:
            print(f"Erro ao processar {symbol}: {e}")

In [ ]:
# Função principal para executar a análise em grupos de symbols usando múltiplas threads
def main():
    # Criar uma instância da exchange (por exemplo, Binance)
    exchange = ccxt.binance()

    # Obter todos os symbols disponíveis na exchange que terminam com 'USDT'
    markets = exchange.load_markets()
    #symbols = [symbol for symbol in markets.keys() if symbol.endswith('USDT')]
    symbols = [symbol for symbol in markets.keys() if symbol == 'ETH/USDT']

    # Definir o tamanho do grupo de symbols por thread
    group_size = 25

    # Dividir a lista de symbols em grupos
    symbol_groups = [symbols[i:i + group_size] for i in range(0, len(symbols), group_size)]

    while True:  # Loop infinito para executar continuamente
        print("Inicio de varredura.")
        with ThreadPoolExecutor(max_workers=len(symbol_groups)) as executor:
            futures = [executor.submit(process_symbols_group, exchange, group) for group in symbol_groups]
            for future in futures:
                future.result()
        
        # Aguardar 5 minutos antes de verificar novamente
        print("Varredura completa. Aguardando próximo ciclo.")
        time.sleep(300)  # 300 segundos = 5 minutos

# Executar a função principal
main()

Inicio de varredura.
Symbol: ETH/USDT, Timestamp antigo: 2024-06-07 16:00:00, Preco antigo: 3687.37, RSI antigo: 28.8145029406287, Timestamp atual: 2024-06-10 00:00:00, Preco atual: 3684.14, RSI atual: 37.41824276400563
Symbol: ETH/USDT, Timestamp antigo: 2024-06-07 16:00:00, Preco antigo: 3687.37, RSI antigo: 28.8145029406287, Timestamp atual: 2024-06-10 00:00:00, Preco atual: 3684.14, RSI atual: 37.41824276400563
Symbol: ETH/USDT, Timestamp antigo: 2024-06-07 16:00:00, Preco antigo: 3687.37, RSI antigo: 28.8145029406287, Timestamp atual: 2024-06-10 04:00:00, Preco atual: 3647.79, RSI atual: 31.197479029629363
Symbol: ETH/USDT, Timestamp antigo: 2024-06-07 16:00:00, Preco antigo: 3687.37, RSI antigo: 28.8145029406287, Timestamp atual: 2024-06-10 00:00:00, Preco atual: 3684.14, RSI atual: 37.41824276400563
Symbol: ETH/USDT, Timestamp antigo: 2024-06-07 16:00:00, Preco antigo: 3687.37, RSI antigo: 28.8145029406287, Timestamp atual: 2024-06-10 04:00:00, Preco atual: 3647.79, RSI atual: 3